## make a new bam2df using mawk

In [1]:
import sys
sys.path.append('./code')
from io import StringIO
from subprocess import Popen, PIPE, run
from HDR_core import run_HDR
from HDR_utils import *

In [ ]:
ls data

### old style

In [4]:
bam_df = bam2df2('data/002_A.filter1.bam', 'chr1')
bam_df[:3]

,Chr,Pos,Seq,Qual,read_len,Soft_start
0,chr1,3631967,ACTCAGCAGAGCGGAGCAAGTGAGCAGGGTGGAGGCCTCCTGACTC...,ADEECFGCGDGGAGDHHDFHDIEHHCHHHDIHEHHHFGGFGIDFGG...,151,0
1,chr1,3632020,GACCTGCGGCTGCTGCAGGACAAAGGCCCAGCGCCTCCAAGGAGCT...,ACDDFGGAGGFHGFHHDHHEFDF:HGHFFDHHBHFGGFDFHHDHHG...,151,0
2,chr1,3632021,ACCTGCGGCTGCTGCAGGACAAAGGCCCAGCGCCTCCAAGGAGCTT...,CCEFIGAHHGIHGIHDHHEFDF-HHIFFDIHCHFG=FDFHIFHHGF...,151,0


In [ ]:
def editbamdf(df):
    '''
    clean_up the bam file
    '''
    # convert position into integer
    df['ref_pos'] = df['ref_pos'].astype(int)
    df['read_len'] = df['seq'].str.len()  # only for debugging
    # extract the intron sizes from the cigar string
    df['Chr_len'] = df['read_len']
    # extraction of soft-clipped bases
    df['soft_start'] = df['cigar'].str.extract(
        r'(^[0-9]+)S').fillna(0).astype(int)
    # filter out reads without a Cell barcode and return only desired columns
    df = df[['name', 'ref_name', 'ref_pos', 'read_len',
             'seq', 'qual', 'soft_start', 'cigar']]
    df.columns = ['name', 'Chr', 'Pos', 'read_len',
                  'Seq', 'Qual', 'Soft_start', 'Cigar']
    return df

In [ ]:
editbamdf(bam_df)[:3]

### new style

In [ ]:
def bam2df2(bam_file, bamtags="", tool='samtools', mut_row=None, region='', q=20):
    '''
    set the region
    '''

    if region:
        mut_pos = region

    elif isinstance(mut_row, pd.Series):
        chrom = mut_row['Chr']
        pos = mut_row['Pos']
        mut_pos = f"{chrom}:{pos}-{pos} "
        print(mut_pos)
        pileup = True
    else:
        mut_pos = ''

    cmd = f"{tool} view {bam_file} {mut_pos} | bam2csv | ./shell/editbam.mawk {q}"
    bam_df = pd.read_csv(StringIO(
        run(cmd, stdout=PIPE, check=True, shell=True).stdout.decode('utf-8')), sep='\t')
    return bam_df

In [ ]:
bam_df2 = bam2df2('data/002_A.filter1.bam', region='chr1')
bam_df2[:3]